In [1]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [2]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 8.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=3c7f1b4c18a09cbd957009fadac956380c8ad195c93b10aa326753b327fdf2d3
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=cd24b107c0e034fb21d5b9f8fa06729d3925d4d0087bdfafbb3749971b9d8bcf
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [3]:
from __future__ import print_function

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

import spacy

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem.lancaster import LancasterStemmer

import operator

np.random.seed(2018)
import sys
import nltk
import csv, os

import pyLDAvis
import pyLDAvis.gensim_models

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

In [6]:
df = pd.read_csv(r'women_salary_neg.csv') 

In [7]:
df['data'] = df.date_comment.tolist()

In [8]:
data = df.date_comment.tolist()

df['data'] = [re.sub('\s*@\s*\s?', '', str(sent)) for sent in df['data']]
df['data'] = [re.sub('\?', '', str(sent)) for sent in df['data']]
df['data'] = [re.sub('\_', '', str(sent)) for sent in df['data']]
df['data'] = [re.sub('@"[\d-]"', '', str(sent)) for sent in df['data']]

#remove new lines characters?
df['data'] = [re.sub('\s+', '', str(sent)) for sent in df['data']]

df['data'] = df['data'].str.lower

<>:3: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \?
<>:5: DeprecationWarning: invalid escape sequence \_
<>:6: DeprecationWarning: invalid escape sequence \d
<>:9: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \?
<>:5: DeprecationWarning: invalid escape sequence \_
<>:6: DeprecationWarning: invalid escape sequence \d
<>:9: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \?
<>:5: DeprecationWarning: invalid escape sequence \_
<>:6: DeprecationWarning: invalid escape sequence \d
<>:9: DeprecationWarning: invalid escape sequence \s
<ipython-input-8-b68628242284>:3: DeprecationWarning: invalid escape sequence \s
  df['data'] = [re.sub('\s*@\s*\s?', '', str(sent)) for sent in df['data']]
<ipython-input-8-b68628242284>:4: DeprecationWar

In [9]:
def sent_to_words(sentences):
  for sentence in sentences :
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))

print(data_words[:3])

[['ended', 'up', 'with', 'exactly', 'the', 'raise', 'asked', 'for'], ['good', 'for', 'you'], ['has', 'always', 'ended', 'up', 'in', 'raise', 'even', 'if', 'itao', 'not', 'as', 'much', 'as', 'had', 'wanted', 'originally', 'still', 'raise']]


In [10]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[2]]])

['has', 'always', 'ended', 'up', 'in', 'raise', 'even', 'if', 'itao', 'not', 'as', 'much', 'as', 'had', 'wanted', 'originally', 'still', 'raise']


/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
def remove_stopwords(texts):
  return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
  return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
  return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [12]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[:3])

['end exactly raise ask %', 'good', 'always end raise even itäó»s as much want originally still raise']


In [13]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)], [(3, 1), (8, 1), (9, 1)]]


In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [18]:
lda_model.print_topics()

[(8,
  '0.001*"previous" + 0.001*"perfect" + 0.001*"resumes" + 0.001*"relative" + 0.001*"rant" + 0.001*"problems" + 0.001*"previously" + 0.001*"sell" + 0.001*"luck" + 0.001*"lots"'),
 (17,
  '0.001*"previous" + 0.001*"perfect" + 0.001*"resumes" + 0.001*"relative" + 0.001*"rant" + 0.001*"problems" + 0.001*"previously" + 0.001*"sell" + 0.001*"luck" + 0.001*"lots"'),
 (29,
  '0.001*"previous" + 0.001*"perfect" + 0.001*"resumes" + 0.001*"relative" + 0.001*"rant" + 0.001*"problems" + 0.001*"previously" + 0.001*"sell" + 0.001*"luck" + 0.001*"lots"'),
 (1,
  '0.001*"you" + 0.001*"the" + 0.001*"that" + 0.001*"is" + 0.001*"it" + 0.001*"and" + 0.001*"salary" + 0.001*"to" + 0.001*"if" + 0.001*"they"'),
 (18,
  '0.027*"you" + 0.027*"good" + 0.027*"for" + 0.001*"sell" + 0.001*"selected" + 0.001*"resumes" + 0.001*"relative" + 0.001*"rant" + 0.001*"problems" + 0.001*"technology"'),
 (24,
  '0.069*"you" + 0.042*"that" + 0.042*"of" + 0.042*"range" + 0.028*"end" + 0.028*"accept" + 0.028*"the" + 0.014*"w

In [19]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=25)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.033337 -0.156540       1        1  32.710841
27     0.006090 -0.211405       2        1  14.223645
28     0.247759 -0.086446       3        1   5.184610
16     0.257153 -0.001730       4        1   5.050856
11     0.111937 -0.211382       5        1   4.890285
22     0.044538 -0.250326       6        1   4.583762
21    -0.265615  0.038455       7        1   4.502000
0      0.198844 -0.150565       8        1   4.047825
26    -0.110267 -0.171028       9        1   3.595496
3     -0.211700 -0.131752      10        1   3.306422
19    -0.239857 -0.032706      11        1   3.045221
7     -0.103339 -0.226609      12        1   2.348619
25    -0.156648 -0.063574      13        1   2.232114
2      0.136729 -0.106632      14        1   2.114575
15    -0.164437  0.173534      15        1   1.943042
5      0.188834  0.076396      16        1   1.341908
6     -0.153122  0.110605      17        1   1.095651
24    -0.095590  0.050399      18        1   0.823959
9      0.081956  0.019194      19        1   0.638460
10     0.099637  0.191434      20        1   0.572583
13    -0.003779  0.018500      21        1   0.500305
20     0.005681  0.202228      22        1   0.498718
14    -0.015754  0.123705      23        1   0.239833
12     0.035696  0.142006      24        1   0.152733
23     0.043838  0.112362      25        1   0.131349
18     0.010235  0.094456      26        1   0.088270
1      0.021010  0.111282      27        1   0.034435
29     0.021170  0.112047      28        1   0.034207
8      0.021170  0.112047      29        1   0.034169
17     0.021170  0.112047      30        1   0.034103, topic_info=          Term        Freq       Total Category  logprob  loglift
9          you  122.000000  122.000000  Default  25.0000  25.0000
5          the  171.000000  171.000000  Default  24.0000  24.0000
65          to  146.000000  146.000000  Default  23.0000  23.0000
78          is   65.000000   65.000000  Default  22.0000  22.0000
93        that   77.000000   77.000000  Default  21.0000  21.0000
..         ...         ...         ...      ...      ...      ...
20  originally    0.001475    2.118669  Topic30  -6.9707   0.7134
21       still    0.001475    3.506647  Topic30  -6.9707   0.2095
22      wanted    0.001475    5.527960  Topic30  -6.9707  -0.2457
23    actually    0.001475    3.979078  Topic30  -6.9707   0.0831
24         and    0.001475  102.391065  Topic30  -6.9707  -3.1646

[1484 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
99        2  0.296822      able
99       15  0.296822      able
99       17  0.296822      able
129       1  0.082994     about
129       2  0.248982     about
...     ...       ...       ...
297       3  0.105126  yourself
297       6  0.210252  yourself
297       7  0.210252  yourself
297      10  0.105126  yourself
681      11  0.744388       yup

[1567 rows x 3 columns], R=25, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 28, 29, 17, 12, 23, 22, 1, 27, 4, 20, 8, 26, 3, 16, 6, 7, 25, 10, 11, 14, 21, 15, 13, 24, 19, 2, 30, 9, 18])